# ngl::Transformation

The ```ngl::Transformation``` class combines scale, translate and rotations around the [Cartesian axis](https://en.wikipedia.org/wiki/Cartesian_coordinate_system) in a set way to get predictable results.  The core method to calculate the matrices are

``` c++
void Transformation::computeMatrices() noexcept
{
  if (!m_isMatrixComputed)       // need to recalculate
  {
    Mat4 scale;
    Mat4 rX;
    Mat4 rY;
    Mat4 rZ;
    Mat4 trans;

    // rotation/scale matrix
    Mat4 rotationScale;
    scale.scale(m_scale.m_x, m_scale.m_y, m_scale.m_z);

    rX.rotateX(m_rotation.m_x);
    rY.rotateY(m_rotation.m_y);
    rZ.rotateZ(m_rotation.m_z);
    rotationScale = scale * rX * rY * rZ;

    // transform matrix
    m_matrix = rotationScale;
    m_matrix.m_m[3][0] = m_position.m_x;
    m_matrix.m_m[3][1] = m_position.m_y;
    m_matrix.m_m[3][2] = m_position.m_z;
    m_matrix.m_m[3][3] = 1;

    // tranpose matrix
    m_transposeMatrix = rotationScale;
    m_transposeMatrix.transpose();
    m_transposeMatrix.m_m[0][3] = m_position.m_x;
    m_transposeMatrix.m_m[1][3] = m_position.m_y;
    m_transposeMatrix.m_m[2][3] = m_position.m_z;
    m_transposeMatrix.m_m[3][3] = 1;

    // inverse matrix
    trans.translate(-m_position.m_x, -m_position.m_y, -m_position.m_z);
    scale.scale(1.0f / m_scale.m_x, 1.0f / m_scale.m_y, 1.0f / m_scale.m_z);
    rX.rotateX(-m_rotation.m_x);
    rY.rotateY(-m_rotation.m_y);
    rZ.rotateZ(-m_rotation.m_z);
    m_inverseMatrix = trans * rZ * rY * rX * scale  ;

    m_isMatrixComputed = true;
  }
}

```

In [1]:
// You may need to modify these paths to suit
#pragma cling add_library_path("$HOME/NGL/lib")
#ifdef __APPLE__
    #pragma cling load("$HOME/NGL/lib/libNGL.dylib")
#else
    #pragma cling load("$HOME/NGL/lib/libNGL.so")
#endif
#pragma cling add_include_path("$HOME/NGL/include")
#pragma cling add_include_path("$HOME/NGL/gl3w")

In [2]:
#include <ngl/Vec3.h>
#include <ngl/Vec4.h>
#include <ngl/Transformation.h>
#include <ngl/NGLStream.h> // for printing
#include <iostream>


In [3]:
{
    ngl::Transformation tx;
    std::cout<<"Default tx \n"<<tx.getMatrix();
    tx.setScale(0.5f,1.0f,0.0f);
    tx.setPosition(-1.0f,2.0f,1.0f);
    tx.setRotation(45.0f,22.0f,12.0f);
    std::cout<<"tx \n"<<tx.getMatrix();
    
}

Default tx 
[1,0,0,0]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]
tx 
[0.453461,0.112083,0,-1]
[0.0963862,0.746728,0,2]
[-0.187303,0.655618,0,1]
[0,0,0,1]


We can reset the transform to the identity matrix using the reset method

In [4]:
{
    ngl::Transformation tx;
    tx.setScale(0.5f,1.0f,0.0f);
    std::cout<<"scale  tx \n"<<tx.getMatrix();
    tx.reset();
    std::cout<<"reset tx \n"<<tx.getMatrix();
    
}

scale  tx 
[0.5,0,0,0]
[0,1,0,0]
[0,0,0,0]
[0,0,0,1]
reset tx 
[1,0,0,0]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]


Transformations can be added as well as set, not the use of an ```ngl::Vec3``` to set which will also take and ```ngl::Vec4``` as will the other set / add methods. 

In [5]:
{
    ngl::Transformation tx;
    for(ngl::Real px=0.0f; px<3.0f; px+=0.5f)
    {
        tx.addPosition(ngl::Vec3(px,0,0));
        std::cout<<px<<'\n'<<tx.getMatrix()<<'\n';
    }
}

0
[1,0,0,0]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]

0.5
[1,0,0,0.5]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]

1
[1,0,0,1.5]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]

1.5
[1,0,0,3]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]

2
[1,0,0,5]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]

2.5
[1,0,0,7.5]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]



It is also possible to set the default matrix in the Transformation, this is useful when passing a Transformation to a generic loadMatrixToShader style function so we don't need to have multiple load methods.

In [8]:
{
    ngl::Mat4 rot;
    rot.rotateX(45.0f);
    ngl::Transformation tx;
    std::cout<<tx.getMatrix()<<'\n';
    tx.setMatrix(rot);
    std::cout<<tx.getMatrix()<<'\n';
}

[1,0,0,0]
[0,1,0,0]
[0,0,1,0]
[0,0,0,1]

[1,0,0,0]
[0,0.906308,-0.422618,0]
[0,0.422618,0.906308,0]
[0,0,0,1]



Usually we want the standard matrix, however it is possible to get the transpose and inverse.

In [7]:
{
    ngl::Transformation tx;
    tx.setPosition(1.0f,2.0f,3.0f);
    tx.setRotation(45.0f,0.0f,1.0f);
    std::cout<<tx.getMatrix()<<'\n';
    std::cout<<tx.getTransposeMatrix()<<'\n';
    std::cout<<tx.getInverseMatrix()<<'\n';
    

}

[0.999848,-0.0123407,0.0123407,1]
[0.0174524,0.706999,-0.706999,2]
[0,0.707107,0.707107,3]
[0,0,0,1]

[0.999848,0.0174524,0,0]
[-0.0123407,0.706999,0.707107,0]
[0.0123407,-0.706999,0.707107,0]
[1,2,3,1]

[0.999848,0.0123407,0.0123407,-1]
[-0.0174524,0.706999,0.706999,-2]
[0,-0.707107,0.707107,-3]
[0,0,0,1]

